<a href="https://colab.research.google.com/github/Shasuf/PathOr/blob/main/bot.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import webbrowser
import platform
import subprocess
import time

def open_webpage(url):
    """
    Opens a webpage in the default browser.
    """
    webbrowser.open(url)

def close_tab():
    """
    Closes the currently active tab in the default browser.
    """
    system = platform.system()
    if system == "Windows":
        subprocess.run(['taskkill', '/F', '/IM', 'chrome.exe'], stdout=subprocess.PIPE, stderr=subprocess.PIPE)
    elif system == "Darwin":
        subprocess.run(['osascript', '-e', 'tell application "Google Chrome" to close active tab'], stdout=subprocess.PIPE, stderr=subprocess.PIPE)
    else:
        print("Unsupported platform for closing tabs.")
def conversational_agent():
    """
    Handles user interactions in a conversational manner.
    """
    print("🤖 Hello! I'm your conversational agent. How can I assist you today?")
    while True:
        user_input = input("You: ").lower()
        if "open" in user_input:
            if "google" in user_input:
                open_webpage("https://www.google.com")
                print("🌐 Google opened in your default browser.")
            else:
                print("I'm sorry, I can only open Google for now.")
        elif "close" in user_input and "tab" in user_input:
            close_tab()
            print("🚪 Closed the tab in your default browser.")
        elif "exit" in user_input or "quit" in user_input:
            print("Goodbye! 👋")
            break
        else:
            print("I'm sorry, I didn't understand that.")
if __name__ == "__main__":
    conversational_agent()


In [1]:
pip install PyPDF2


In [2]:
import nltk
nltk.download('punkt')


[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [3]:
import nltk
nltk.download('stopwords')


[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [4]:
import PyPDF2

def extract_text_from_pdf(pdf_path):
    """
    Extracts text from a PDF file.

    Args:
    - pdf_path (str): Path to the PDF file.

    Returns:
    - text (str): Extracted text from the PDF.
    """
    text = ""
    with open(pdf_path, "rb") as file:
        reader = PyPDF2.PdfReader(file)
        for page_num in range(len(reader.pages)):
            page = reader.pages[page_num]
            text += page.extract_text()
    return text

# Example usage:
pdf_text = extract_text_from_pdf("/content/sharfa.pdf")
print(pdf_text[:500])  # Print the first 500 characters of the extracted text


Dear Hiring Manager, I am writing to express my interest in the Data Scientist  position at, as advertised. With a strong background in developing cutting-edge AI solutions and a track record of driving innovation, I am excited about the opportunity to contribute to your team and help advance the company's mission. In my previous role as a Computer Vision and Machine learning Engineer, I had the privilege of collaborating closely with senior de-velopers and engineers to develop real-time compute


In [5]:
import re
import string
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize

def preprocess_text(text):
    """
    Preprocesses the text by tokenizing, lowercasing, removing punctuation,
    and stop words.

    Args:
    - text (str): Input text.

    Returns:
    - preprocessed_text (str): Preprocessed text.
    """
    # Tokenize the text
    tokens = word_tokenize(text)

    # Lowercase and remove punctuation
    tokens = [token.lower() for token in tokens if token not in string.punctuation]

    # Remove stop words
    stop_words = set(stopwords.words('english'))
    tokens = [token for token in tokens if token not in stop_words]

    # Join tokens back into a string
    preprocessed_text = ' '.join(tokens)

    return preprocessed_text

def preprocess_question(question):
    """
    Preprocesses the question by lowercasing and removing punctuation.

    Args:
    - question (str): Input question.

    Returns:
    - preprocessed_question (str): Preprocessed question.
    """
    # Lowercase and remove punctuation
    question = re.sub(r'[^\w\s]', '', question.lower())
    return question

# Example usage:
preprocessed_pdf_text = preprocess_text(pdf_text)
preprocessed_question = preprocess_question("What is the main topic of the PDF?")
print(preprocessed_pdf_text[:500])  # Print the first 500 characters of the preprocessed text
print(preprocessed_question)  # Print the preprocessed question


dear hiring manager writing express interest data scientist position advertised strong background developing cutting-edge ai solutions track record driving innovation excited opportunity contribute team help advance company 's mission previous role computer vision machine learning engineer privilege collaborating closely senior de-velopers engineers develop real-time computer vision machine learning solutions using iot devices smart cameras experience honed ability tackle complex challenges deli
what is the main topic of the pdf


In [6]:
import re
import string
import math
from collections import Counter

def tokenize(text):
    """
    Tokenizes the text by splitting on whitespace and removing punctuation.

    Args:
    - text (str): Input text.

    Returns:
    - tokens (list): List of tokens.
    """
    tokens = re.findall(r'\b\w+\b', text.lower())
    return tokens

def compute_tf(tokens):
    """
    Computes the term frequency (TF) for each token in the list of tokens.

    Args:
    - tokens (list): List of tokens.

    Returns:
    - tf (dict): Dictionary containing the TF for each token.
    """
    tf = Counter(tokens)
    total_tokens = len(tokens)
    for token in tf:
        tf[token] /= total_tokens
    return tf

def compute_idf(docs):
    """
    Computes the inverse document frequency (IDF) for each token in the list of documents.

    Args:
    - docs (list of list): List of documents, where each document is represented as a list of tokens.

    Returns:
    - idf (dict): Dictionary containing the IDF for each token.
    """
    idf = {}
    num_docs = len(docs)
    for doc in docs:
        doc_set = set(doc)
        for token in doc_set:
            idf[token] = idf.get(token, 0) + 1
    for token, freq in idf.items():
        idf[token] = math.log(num_docs / (freq + 1))
    return idf

def compute_tfidf(tf, idf):
    """
    Computes the TF-IDF score for each token given the TF and IDF values.

    Args:
    - tf (dict): Dictionary containing the TF for each token.
    - idf (dict): Dictionary containing the IDF for each token.

    Returns:
    - tfidf (dict): Dictionary containing the TF-IDF score for each token.
    """
    tfidf = {}
    for token, tf_value in tf.items():
        tfidf[token] = tf_value * idf[token]
    return tfidf

def preprocess_and_compute_tfidf(documents):
    """
    Preprocesses the documents (PDF text and question) and computes TF-IDF vectors.

    Args:
    - documents (list of str): List of documents, where each document is represented as a string.

    Returns:
    - tfidf_vectors (list of dict): List of TF-IDF vectors for each document.
    """
    tokenized_documents = [tokenize(doc) for doc in documents]
    tf_documents = [compute_tf(doc) for doc in tokenized_documents]
    idf = compute_idf(tokenized_documents)
    tfidf_vectors = [compute_tfidf(tf_doc, idf) for tf_doc in tf_documents]
    return tfidf_vectors




In [7]:
# Example usage:
pdf_text = "Dear Hiring Manager, I am writing to express my interest in the Data Scientist position..."
question = "What is the main topic of the PDF?"
documents = [pdf_text, question]
tfidf_vectors = preprocess_and_compute_tfidf(documents)

# Print the TF-IDF vectors
print(tfidf_vectors)



[{'dear': 0.0, 'hiring': 0.0, 'manager': 0.0, 'i': 0.0, 'am': 0.0, 'writing': 0.0, 'to': 0.0, 'express': 0.0, 'my': 0.0, 'interest': 0.0, 'in': 0.0, 'the': -0.027031007207210963, 'data': 0.0, 'scientist': 0.0, 'position': 0.0}, {'what': 0.0, 'is': 0.0, 'the': -0.10136627702704111, 'main': 0.0, 'topic': 0.0, 'of': 0.0, 'pdf': 0.0}]


In [9]:
import re
import string
import math
import numpy as np
from collections import Counter

def tokenize(text):
    """
    Tokenizes the text by splitting on whitespace and removing punctuation.

    Args:
    - text (str): Input text.

    Returns:
    - tokens (list): List of tokens.
    """
    tokens = re.findall(r'\b\w+\b', text.lower())
    return tokens

def compute_tf(tokens):
    """
    Computes the term frequency (TF) for each token in the list of tokens.

    Args:
    - tokens (list): List of tokens.

    Returns:
    - tf (dict): Dictionary containing the TF for each token.
    """
    tf = Counter(tokens)
    total_tokens = len(tokens)
    for token in tf:
        tf[token] /= total_tokens
    return tf

def compute_idf(docs):
    """
    Computes the inverse document frequency (IDF) for each token in the list of documents.

    Args:
    - docs (list of list): List of documents, where each document is represented as a list of tokens.

    Returns:
    - idf (dict): Dictionary containing the IDF for each token.
    """
    idf = {}
    num_docs = len(docs)
    for doc in docs:
        doc_set = set(doc)
        for token in doc_set:
            idf[token] = idf.get(token, 0) + 1
    for token, freq in idf.items():
        idf[token] = math.log(num_docs / (freq + 1))
    return idf

def compute_idf(docs):
    """
    Computes the inverse document frequency (IDF) for each token in the list of documents.

    Args:
    - docs (list of list): List of documents, where each document is represented as a list of tokens.

    Returns:
    - idf (dict): Dictionary containing the IDF for each token.
    """
    idf = {}
    num_docs = len(docs)
    for doc in docs:
        doc_set = set(doc)
        for token in doc_set:
            idf[token] = idf.get(token, 0) + 1
    for token, freq in idf.items():
        idf[token] = math.log(num_docs / freq)  # Fix the IDF calculation
    return idf


def preprocess_and_compute_tfidf(documents):
    """
    Preprocesses the documents (PDF text and question) and computes TF-IDF vectors.

    Args:
    - documents (list of str): List of documents, where each document is represented as a string.

    Returns:
    - tfidf_vectors (list of dict): List of TF-IDF vectors for the PDF text and the question.
    """
    tokenized_documents = [tokenize(doc) for doc in documents]
    tf_documents = [compute_tf(doc) for doc in tokenized_documents]
    idf = compute_idf(tokenized_documents)
    tfidf_vectors = [compute_tfidf(tf_doc, idf) for tf_doc in tf_documents]

    # Ensure tfidf_vectors is in the expected format
    assert len(tfidf_vectors) == 2, "Expected two TF-IDF vectors, but found {}".format(len(tfidf_vectors))
    for i, tfidf_vector in enumerate(tfidf_vectors):
        if not isinstance(tfidf_vector, dict):
            # If TF-IDF vector is not a dictionary, create an empty one
            print("Warning: TF-IDF vector {} is not a dictionary. Creating an empty one.".format(i + 1))
            tfidf_vectors[i] = {}

    return tfidf_vectors

def compute_cosine_similarity(tfidf_vector1, tfidf_vector2):
    """
    Computes the cosine similarity between two TF-IDF vectors.

    Args:
    - tfidf_vector1 (dict): TF-IDF vector 1.
    - tfidf_vector2 (dict): TF-IDF vector 2.

    Returns:
    - cosine_similarity (float): Cosine similarity between the two TF-IDF vectors.
    """
    if not isinstance(tfidf_vector1, dict) or not isinstance(tfidf_vector2, dict):
        print("Error: TF-IDF vectors are not properly computed.")
        return None

    tokens = set(tfidf_vector1.keys()) | set(tfidf_vector2.keys())
    dot_product = sum(tfidf_vector1.get(token, 0) * tfidf_vector2.get(token, 0) for token in tokens)
    norm1 = np.linalg.norm(list(tfidf_vector1.values()))
    norm2 = np.linalg.norm(list(tfidf_vector2.values()))
    cosine_similarity = dot_product / (norm1 * norm2)
    return cosine_similarity

def retrieve_relevant_passages(tfidf_vectors, threshold=0.5):
    """
    Retrieves relevant passages from the PDF based on cosine similarity with the question.

    Args:
    - tfidf_vectors (list of dict): List of TF-IDF vectors for the PDF text and the question.
    - threshold (float): Similarity threshold for considering a passage as relevant. Default is 0.5.

    Returns:
    - relevant_passages (list of str): List of relevant passages from the PDF.
    """
    question_tfidf_vector, pdf_tfidf_vector = tfidf_vectors

    if question_tfidf_vector is None or pdf_tfidf_vector is None:
        print("Error: TF-IDF vectors are not properly computed.")
        return []

    relevant_passages = []
    for passage, passage_tfidf_vector in pdf_tfidf_vector.items():
        similarity = compute_cosine_similarity(question_tfidf_vector, passage_tfidf_vector)
        if similarity is not None and similarity >= threshold:
            relevant_passages.append(passage)
    return relevant_passages

# Example usage:
pdf_text = "Dear Hiring Manager, I am writing to express my interest in the Data Scientist position..."
question = "What is the main topic of the PDF?"
documents = [pdf_text, question]
tfidf_vectors = preprocess_and_compute_tfidf(documents)
print("TF-IDF Vectors:", tfidf_vectors)  # Add this line to inspect tfidf_vectors
relevant_passages = retrieve_relevant_passages(tfidf_vectors)
print("Relevant Passages:", relevant_passages)


TF-IDF Vectors: [{'dear': 0.046209812037329684, 'hiring': 0.046209812037329684, 'manager': 0.046209812037329684, 'i': 0.046209812037329684, 'am': 0.046209812037329684, 'writing': 0.046209812037329684, 'to': 0.046209812037329684, 'express': 0.046209812037329684, 'my': 0.046209812037329684, 'interest': 0.046209812037329684, 'in': 0.046209812037329684, 'the': 0.0, 'data': 0.046209812037329684, 'scientist': 0.046209812037329684, 'position': 0.046209812037329684}, {'what': 0.08664339756999316, 'is': 0.08664339756999316, 'the': 0.0, 'main': 0.08664339756999316, 'topic': 0.08664339756999316, 'of': 0.08664339756999316, 'pdf': 0.08664339756999316}]
Error: TF-IDF vectors are not properly computed.
Error: TF-IDF vectors are not properly computed.
Error: TF-IDF vectors are not properly computed.
Error: TF-IDF vectors are not properly computed.
Error: TF-IDF vectors are not properly computed.
Error: TF-IDF vectors are not properly computed.
Error: TF-IDF vectors are not properly computed.
Relevant 

In [10]:
import re

def tokenize(text):
    """
    Tokenizes the text by splitting on whitespace and removing punctuation.

    Args:
    - text (str): Input text.

    Returns:
    - tokens (list): List of tokens.
    """
    tokens = re.findall(r'\b\w+\b', text.lower())
    print("Tokens:", tokens)
    return tokens


In [11]:
from collections import Counter

def compute_tf(tokens):
    """
    Computes the term frequency (TF) for each token in the list of tokens.

    Args:
    - tokens (list): List of tokens.

    Returns:
    - tf (dict): Dictionary containing the TF for each token.
    """
    tf = Counter(tokens)
    total_tokens = len(tokens)
    for token in tf:
        tf[token] /= total_tokens
    print("TF:", tf)
    return tf


In [12]:
import math

def compute_idf(docs):
    """
    Computes the inverse document frequency (IDF) for each token in the list of documents.

    Args:
    - docs (list of list): List of documents, where each document is represented as a list of tokens.

    Returns:
    - idf (dict): Dictionary containing the IDF for each token.
    """
    idf = {}
    num_docs = len(docs)
    for doc in docs:
        doc_set = set(doc)
        for token in doc_set:
            idf[token] = idf.get(token, 0) + 1
    for token, freq in idf.items():
        idf[token] = math.log(num_docs / (freq + 1))
    print("IDF:", idf)
    return idf


In [13]:
import math

def compute_idf(docs):
    """
    Computes the inverse document frequency (IDF) for each token in the list of documents.

    Args:
    - docs (list of list): List of documents, where each document is represented as a list of tokens.

    Returns:
    - idf (dict): Dictionary containing the IDF for each token.
    """
    idf = {}
    num_docs = len(docs)
    for doc in docs:
        doc_set = set(doc)
        for token in doc_set:
            idf[token] = idf.get(token, 0) + 1
    for token, freq in idf.items():
        idf[token] = math.log(num_docs / (freq + 1))
    print("IDF:", idf)
    return idf


In [14]:
def compute_tfidf(tf, idf):
    """
    Computes the TF-IDF score for each token given the TF and IDF values.

    Args:
    - tf (dict): Dictionary containing the TF for each token.
    - idf (dict): Dictionary containing the IDF for each token.

    Returns:
    - tfidf (dict): Dictionary containing the TF-IDF score for each token.
    """
    tfidf = {token: tf[token] * idf[token] for token in tf}
    print("TF-IDF:", tfidf)
    return tfidf


In [15]:
def preprocess_and_compute_tfidf(documents):
    """
    Preprocesses the documents (PDF text and question) and computes TF-IDF vectors.

    Args:
    - documents (list of str): List of documents, where each document is represented as a string.

    Returns:
    - tfidf_vectors (list of dict): List of TF-IDF vectors for the PDF text and the question.
    """
    tokenized_documents = [tokenize(doc) for doc in documents]
    tf_documents = [compute_tf(doc) for doc in tokenized_documents]
    idf = compute_idf(tokenized_documents)
    tfidf_vectors = [compute_tfidf(tf_doc, idf) for tf_doc in tf_documents]

    # Ensure tfidf_vectors is in the expected format
    assert len(tfidf_vectors) == 2, "Expected two TF-IDF vectors, but found {}".format(len(tfidf_vectors))
    for i, tfidf_vector in enumerate(tfidf_vectors):
        if not isinstance(tfidf_vector, dict):
            # If TF-IDF vector is not a dictionary, create an empty one
            print("Warning: TF-IDF vector {} is not a dictionary. Creating an empty one.".format(i + 1))
            tfidf_vectors[i] = {}

    return tfidf_vectors


In [16]:
pdf_text = "Dear Hiring Manager, I am writing to express my interest in the Data Scientist position..."
question = "What is the main topic of the PDF?"
documents = [pdf_text, question]
tfidf_vectors = preprocess_and_compute_tfidf(documents)
print("TF-IDF Vectors:", tfidf_vectors)


Tokens: ['dear', 'hiring', 'manager', 'i', 'am', 'writing', 'to', 'express', 'my', 'interest', 'in', 'the', 'data', 'scientist', 'position']
Tokens: ['what', 'is', 'the', 'main', 'topic', 'of', 'the', 'pdf']
TF: Counter({'dear': 0.06666666666666667, 'hiring': 0.06666666666666667, 'manager': 0.06666666666666667, 'i': 0.06666666666666667, 'am': 0.06666666666666667, 'writing': 0.06666666666666667, 'to': 0.06666666666666667, 'express': 0.06666666666666667, 'my': 0.06666666666666667, 'interest': 0.06666666666666667, 'in': 0.06666666666666667, 'the': 0.06666666666666667, 'data': 0.06666666666666667, 'scientist': 0.06666666666666667, 'position': 0.06666666666666667})
TF: Counter({'the': 0.25, 'what': 0.125, 'is': 0.125, 'main': 0.125, 'topic': 0.125, 'of': 0.125, 'pdf': 0.125})
IDF: {'writing': 0.0, 'i': 0.0, 'am': 0.0, 'dear': 0.0, 'express': 0.0, 'position': 0.0, 'manager': 0.0, 'data': 0.0, 'the': -0.40546510810816444, 'to': 0.0, 'interest': 0.0, 'my': 0.0, 'scientist': 0.0, 'hiring': 0.0,

In [ ]:
pip install fitz

In [ ]:
pip install frontend

## The Actual code after all testing


In [29]:
import re
import string
import math
import numpy as np
from collections import Counter
from PyPDF2 import PdfReader

def extract_text_from_pdf(pdf_file):
    """
    Extracts text from a PDF file.

    Args:
    - pdf_file (str): Path to the PDF file.

    Returns:
    - text (str): Extracted text from the PDF.
    """
    try:
        with open(pdf_file, "rb") as file:
            reader = PdfReader(file)
            text = ""
            for page in reader.pages:
                text += page.extract_text()
        return text
    except Exception as e:
        print("An error occurred:", e)
        return None

def tokenize(text):
    """
    Tokenizes the text by splitting on whitespace and removing punctuation.

    Args:
    - text (str): Input text.

    Returns:
    - tokens (list): List of tokens.
    """
    tokens = re.findall(r'\b\w+\b', text.lower())
    return tokens

def compute_tf(tokens):
    """
    Computes the term frequency (TF) for each token in the list of tokens.

    Args:
    - tokens (list): List of tokens.

    Returns:
    - tf (dict): Dictionary containing the TF for each token.
    """
    tf = Counter(tokens)
    total_tokens = len(tokens)
    for token in tf:
        tf[token] /= total_tokens
    return tf

def compute_idf(docs):
    """
    Computes the inverse document frequency (IDF) for each token in the list of documents.

    Args:
    - docs (list of list): List of documents, where each document is represented as a list of tokens.

    Returns:
    - idf (dict): Dictionary containing the IDF for each token.
    """
    idf = {}
    num_docs = len(docs)
    for doc in docs:
        doc_set = set(doc)
        for token in doc_set:
            idf[token] = idf.get(token, 0) + 1
    for token, freq in idf.items():
        idf[token] = math.log(num_docs / freq)  # Divide by total number of documents
    return idf


def compute_tfidf(tf, idf):
    """
    Computes the TF-IDF score for each token given the TF and IDF values.

    Args:
    - tf (dict): Dictionary containing the TF for each token.
    - idf (dict): Dictionary containing the IDF for each token.

    Returns:
    - tfidf (dict): Dictionary containing the TF-IDF score for each token.
    """
    tfidf = {}
    for token, tf_value in tf.items():
        tfidf[token] = tf_value * idf[token]
    return tfidf

def preprocess_and_compute_tfidf(documents):
    """
    Preprocesses the documents (PDF text and question) and computes TF-IDF vectors.

    Args:
    - documents (list of str): List of documents, where each document is represented as a string.

    Returns:
    - tfidf_vectors (list of dict): List of TF-IDF vectors for the PDF text and the question.
    """
    tokenized_documents = [tokenize(doc) for doc in documents]
    print("Tokens:", tokenized_documents)
    tf_documents = [compute_tf(doc) for doc in tokenized_documents]
    print("TF:", tf_documents)
    idf = compute_idf(tokenized_documents)
    print("IDF:", idf)
    tfidf_vectors = [compute_tfidf(tf_doc, idf) for tf_doc in tf_documents]
    print("TF-IDF:", tfidf_vectors)

    # Ensure tfidf_vectors is in the expected format
    assert len(tfidf_vectors) == 2, "Expected two TF-IDF vectors, but found {}".format(len(tfidf_vectors))
    for i, tfidf_vector in enumerate(tfidf_vectors):
        if not isinstance(tfidf_vector, dict):
            # If TF-IDF vector is not a dictionary, create an empty one
            print("Warning: TF-IDF vector {} is not a dictionary. Creating an empty one.".format(i + 1))
            tfidf_vectors[i] = {}

    return tfidf_vectors

def compute_cosine_similarity(tfidf_vector1, tfidf_vector2):
    """
    Computes the cosine similarity between two TF-IDF vectors.

    Args:
    - tfidf_vector1 (dict): TF-IDF vector 1.
    - tfidf_vector2 (dict): TF-IDF vector 2.

    Returns:
    - cosine_similarity (float): Cosine similarity between the two TF-IDF vectors.
    """
    if not isinstance(tfidf_vector1, dict) or not isinstance(tfidf_vector2, dict):
        print("Error: TF-IDF vectors are not properly computed.")
        return None

    tokens = set(tfidf_vector1.keys()) | set(tfidf_vector2.keys())
    dot_product = sum(tfidf_vector1.get(token, 0) * tfidf_vector2.get(token, 0) for token in tokens)
    norm1 = np.linalg.norm(list(tfidf_vector1.values()))
    norm2 = np.linalg.norm(list(tfidf_vector2.values()))
    cosine_similarity = dot_product / (norm1 * norm2)
    return cosine_similarity

def retrieve_relevant_passages(tfidf_vectors, threshold=0.5):
    """
    Retrieves relevant passages from the PDF based on cosine similarity with the question.

    Args:
    - tfidf_vectors (list of dict): List of TF-IDF vectors for the PDF text and the question.
    - threshold (float): Similarity threshold for considering a passage as relevant. Default is 0.5.

    Returns:
    - relevant_passages (list of str): List of relevant passages from the PDF.
    """
    question_tfidf_vector, pdf_tfidf_vector = tfidf_vectors

    if question_tfidf_vector is None or pdf_tfidf_vector is None:
        print("Error: TF-IDF vectors are not properly computed.")
        return []

    relevant_passages = []
    for passage, passage_tfidf_vector in pdf_tfidf_vector.items():
        similarity = compute_cosine_similarity(question_tfidf_vector, passage_tfidf_vector)
        if similarity is not None and similarity >= threshold:
            relevant_passages.append(passage)
    return relevant_passages

def answer_question(pdf_file, question, threshold=0.5):
    """
    Answers a question based on the content of a PDF file.

    Args:
    - pdf_file (str): Path to the PDF file.
    - question (str): The question to be answered.
    - threshold (float): Similarity threshold for considering a passage as relevant. Default is 0.5.

    Returns:
    - answer (str or None): The answer to the question, or None if no relevant passages are found.
    """
    # Extract text from PDF
    pdf_text = extract_text_from_pdf(pdf_file)
    if pdf_text is None:
        print("Error: Failed to extract text from the PDF.")
        return None

    # Preprocess PDF text and question, and compute TF-IDF vectors
    documents = [pdf_text, question]
    tfidf_vectors = preprocess_and_compute_tfidf(documents)
    if not tfidf_vectors:
        print("Error: TF-IDF vectors are not properly computed.")
        return None

    # Retrieve relevant passages from the PDF
    relevant_passages = retrieve_relevant_passages(tfidf_vectors, threshold)
    if not relevant_passages:
        print("No relevant passages found.")
        return None

    # Concatenate relevant passages as the answer
    answer = ' '.join(relevant_passages)
    return answer

# Example usage:
pdf_file = "/content/sharfa.pdf"
question = "What is qualification?"
answer = answer_question(pdf_file, question)
print("Answer:", answer)


Tokens: [['dear', 'hiring', 'manager', 'i', 'am', 'writing', 'to', 'express', 'my', 'interest', 'in', 'the', 'data', 'scientist', 'position', 'at', 'as', 'advertised', 'with', 'a', 'strong', 'background', 'in', 'developing', 'cutting', 'edge', 'ai', 'solutions', 'and', 'a', 'track', 'record', 'of', 'driving', 'innovation', 'i', 'am', 'excited', 'about', 'the', 'opportunity', 'to', 'contribute', 'to', 'your', 'team', 'and', 'help', 'advance', 'the', 'company', 's', 'mission', 'in', 'my', 'previous', 'role', 'as', 'a', 'computer', 'vision', 'and', 'machine', 'learning', 'engineer', 'i', 'had', 'the', 'privilege', 'of', 'collaborating', 'closely', 'with', 'senior', 'de', 'velopers', 'and', 'engineers', 'to', 'develop', 'real', 'time', 'computer', 'vision', 'and', 'machine', 'learning', 'solutions', 'using', 'iot', 'devices', 'and', 'smart', 'cameras', 'this', 'experience', 'has', 'honed', 'my', 'ability', 'to', 'tackle', 'complex', 'challenges', 'and', 'deliver', 'scalable', 'solutions', 